In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/enis/projects/nna/src


In [2]:
from fileUtils import list_files,getLength,read_file_properties_v2

from pathlib import Path
import pandas as pd
import datetime


In [20]:
# PARAMETERS

# where to save txt file storing length info
data_folder="/home/enis/projects/nna/data/"

# path to search for audio files
# where
# ignore_folders=['/tank/data/nna/real/stinchcomb/']
# search_path="/tank/data/nna/real/"

ignore_folders=["/tank/data/nna/real/stinchcomb/dups/","/tank/data/nna/real/stinchcomb/excerpts/"]
search_path="/tank/data/nna/real/stinchcomb/"


# if we already have a list of files we can load them 
files_list_path=data_folder+"stinchcomb_files_pathV1.txt"
# if we calculated audio lengths and saved them into text file, 
# we can load them
fileswlen_path = data_folder+"stinchcomb_files_wlenV1.txt"

# do NOT add pkl at the end
pkl_file_name="stinchcomb_dataV1"



In [14]:
# Find files
# in given search path ignoring given directories
if not Path(fileswlen_path).exists():
    files_path_list=list_files(search_path,ignore_folders)
else:
    with open(files_list_path,"r") as f:
        lines=f.readlines()
        files_path_list=[line.strip() for line in lines]


In [18]:
# Load or calculate Audio length

# learn length of each audio and store in a text file, 
# if file already exists, it tries to get data from there
if not Path(fileswlen_path).exists():
    length_dict={}
    for f in files_path_list:
        length=float(getLength(f))
        length_dict[f]=length

    length_list=list(length_dict.items())
    with open(fileswlen_path,"w") as f:
        for line in length_list:
            f.write(",".join([line[0],str(line[1])])+"\n")

with open(fileswlen_path,"r") as f:
    lines=f.readlines()
    fileswlen=[line.strip().split(",") for line in lines]

In [19]:
# turn results into a dict
fileswlen=dict([(i[0],float(i[1])) for i in fileswlen])

In [25]:
file_properties,exceptions=read_file_properties_v2(files_path_list)
for f,lengthSeconds in fileswlen.items():
    file_properties[Path(f)]["durationSec"] = lengthSeconds
    file_properties[Path(f)]["timestampEnd"] = file_properties[Path(f)]["timestamp"] + datetime.timedelta(seconds=lengthSeconds)
file_properties_df=pd.DataFrame(file_properties).T


In [36]:
file_properties_df.to_pickle(data_folder+pkl_file_name+".pkl")

In [37]:
# file_properties_df_test=pd.read_pickle(data_folder+pkl_file_name+".pkl")